In [1]:
from utils import ProgressBar
from collections import Counter, defaultdict
sentlength = 10
enlines = []
START = '<START>'
END = '<END>'

pb = ProgressBar(worksum=10000000)
pb.startjob()
num = 0
# 只取英文句子
with open('data/segmented_train_seg_by_word.txt') as fhdl:
    for line in fhdl:
        num += 1
        # 使用行号作为标注，奇数行为英文，偶数行为中文
        if num % 2 == 1:
            enline = line
        else:
            continue
        
        enlinesp = [START, START] + [i.lower() for i in enline.strip("\n").split()] + [END, END]
        # 设置一个最大长度提升demo速度
        if len(enlinesp) <= sentlength:
            enlines.append(enlinesp)
        if (num // 2) % 1000 == 0:
            pb.complete(1000)

 100.00 % [==================================================>] 10000000/10000000 	 used:40s eta:0 s

In [2]:
model = defaultdict(lambda: defaultdict(lambda: 0))

# 统计3-grams
for line in enlines:
    for i in range(len(line)-2):
        w1 = line[i]
        w2 = line[i+1]
        w3 = line[i+2]
        
        model[(w1, w2)][w3] += 1

# 把计数转换成概率
for prefix in model:
    total_count = float(sum(model[prefix].values()))
    for word in model[prefix]:
        model[prefix][word] /= total_count

In [3]:
import random


def generate_a_sentence(model):
    text = ['<START>', '<START>']
    sentence_finished = False

    while not sentence_finished:
        # 设定一个随机阈值 (增加文本多样性)
        r = random.random()
        accumulator = .0

        for word in model[tuple(text[-2:])].keys():
            accumulator += model[tuple(text[-2:])][word]
            # 当累加概率超过阈值时，选择该词汇
            if accumulator >= r:
                if word == '<END>':
                    sentence_finished = True
                else:
                    text.append(word)
                break

    sentence = ' '.join([t for t in text[2:] if t])
    return sentence

In [4]:
# 生成10个句子
for i in range(10):
    sent = generate_a_sentence(model)
    print(sent)

he is my future .
will do .
reliable ,
who ' d be invisible . visible -
i get sunburn
but that day cannot come over .
hhello , come behind me
listen , what an accomplishment
she ' s so bossy .
i feel so sad .
